In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.optim import Adam, AdamW

from pathlib import Path

from tqdm import tqdm

BASE_PATH = Path("data/2d/")
DATA_PATH = Path(BASE_PATH / "point_cloud_samples")
MODEL_PATH = Path(BASE_PATH / "models" / "simple_tanh")

In [ ]:
dim = 2
files = list(DATA_PATH.glob(f"*.csv"))
N_EPOCHS = 5000

for file in tqdm(files):
    fig, ax = plt.subplots(figsize=(10, 4))
    for rep in range(1, 5):
        data = pd.read_csv(file)
        data = data.to_numpy()
        model = nn.Sequential(
                    nn.Linear(dim, 512),
                    nn.Tanh(),
                    nn.Linear(512, 1),
                    nn.Sigmoid()
                    )
        optimizer = AdamW(model.parameters(), lr=0.001)

        losses = np.zeros(N_EPOCHS)
        for epoch in range(N_EPOCHS):
            model.train()
            optimizer.zero_grad()
            output = model.forward(torch.Tensor(data[:, 0:dim])) #use dim for x and y column separation
            loss = nn.functional.mse_loss(output, target = torch.Tensor(data[:, dim]).unsqueeze(1)) 
            loss.backward()
            optimizer.step()
            losses[epoch] = loss 
        print(loss)
        ax.plot(losses, label=f"Run {rep}")
        #pd.DataFrame(losses).to_csv(Path(LOSS_PATH / (file.stem + f"_NNrep_{rep}_loss_NEpochs{N_EPOCHS}.csv")))
        #torch.save(model.state_dict(), Path(MODEL_PATH / (file.stem + f"_NNrep_{rep}_NEpochs{N_EPOCHS}.pt")))
    ax.set_yscale("log")
    ax.legend()
    ax.set_title(file.stem)
    fig.show()